In [1]:
import sys
import random
import string
import os
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel, QPushButton, QComboBox, QSlider, QMessageBox
from PyQt5.QtCore import Qt

from password_strength import PasswordStats

class PasswordManager(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()
        
    def initUI(self):
        self.setWindowTitle('Password Manager')
        layout = QVBoxLayout()

        # Create widgets
        self.total_characters_label = QLabel('Total Characters:')
        self.total_characters_slider = QSlider(Qt.Horizontal)
        self.total_characters_slider.setMinimum(1)
        self.total_characters_slider.setMaximum(50)  # Maximum number of characters changed to 50
        self.total_characters_slider.setValue(1)  # Initial value of the slider
        self.total_characters_display = QLabel('1')  # Initial value of the total characters
        self.special_characters_label = QLabel('Number of Special Characters:')
        self.special_characters_input = QComboBox()  # Dropdown for selecting number of special characters
        self.special_characters_input.addItems([str(i) for i in range(0, 11)])  # Options 0-10
        self.number_label = QLabel('Number of Numbers:')
        self.number_input = QComboBox()  # Dropdown for selecting number of numbers
        self.number_input.addItems([str(i) for i in range(0, 11)])  # Options 0-10
        self.capital_letters_label = QLabel('Include Capital Letters?')
        self.capital_letters_input = QComboBox()
        self.capital_letters_input.addItems(['Yes', 'No'])
        self.generate_button = QPushButton('Generate Password')
        self.strength_label = QLabel('Password Strength:')
        self.password_label = QLabel('Generated Password: ')

        # Add widgets to layout
        layout.addWidget(self.total_characters_label)
        layout.addWidget(self.total_characters_slider)
        layout.addWidget(self.total_characters_display)
        layout.addWidget(self.special_characters_label)
        layout.addWidget(self.special_characters_input)
        layout.addWidget(self.number_label)
        layout.addWidget(self.number_input)
        layout.addWidget(self.capital_letters_label)
        layout.addWidget(self.capital_letters_input)
        layout.addWidget(self.generate_button)
        layout.addWidget(self.strength_label)
        layout.addWidget(self.password_label)

        # Connect slider value changed to update_display function
        self.total_characters_slider.valueChanged.connect(self.update_total_characters_display)

        # Connect button click to generate_password function
        self.generate_button.clicked.connect(self.generate_password)

        # Set layout
        self.setLayout(layout)

    def update_total_characters_display(self, value):
        # Update the label to display the current value of the total characters slider
        self.total_characters_display.setText(str(value))

    def generate_password(self):
        total_characters = int(self.total_characters_display.text())  # Convert the displayed value to an integer
        special_characters_count = int(self.special_characters_input.currentText())
        number_count = int(self.number_input.currentText())
        allow_special_chars = special_characters_count > 0
        allow_numbers = number_count > 0
        include_capital_letters = self.capital_letters_input.currentText() == 'Yes'

        if special_characters_count + number_count > total_characters:
            QMessageBox.warning(self, 'Error', 'Number of special characters and numbers cannot exceed total characters!')
            return

        # Generate password
        password = self.generate_password_helper(total_characters, special_characters_count, number_count, allow_special_chars, allow_numbers, include_capital_letters)

        # Display the generated password
        self.password_label.setText(f'Generated Password: {password}')

        # Evaluate password strength
        strength_score = PasswordStats(password).strength() * 100
        self.strength_label.setText(f'Password Strength: {strength_score:.2f} / 100')

    def generate_password_helper(self, length, special_characters_count, number_count, allow_special_chars, allow_numbers, include_capital_letters):
        file_path = os.path.join(os.path.expanduser('~'), 'Documents', 'Password', 'password.lst')
        with open(file_path, 'r') as file:
            words = [line.strip() for line in file]

        length -= special_characters_count + number_count
        
        password = ''.join(random.choices(words, k=length))
        
        # Ensure that the password length matches the desired length
        password = password[:length]

        # Maintain a set of used indexes for special characters
        used_indexes = set()

        # Generate the special characters and insert them at random positions in the password
        if allow_special_chars:
            for _ in range(special_characters_count):
                special_char = random.choice(string.punctuation)
                insert_index = random.randint(0, length - 1)  # Generate a random index within the valid range
                while insert_index in used_indexes:
                    insert_index = random.randint(0, length - 1)  # Generate a new random index if already used
                password = password[:insert_index] + special_char + password[insert_index:]
                used_indexes.add(insert_index)

        # Generate the numbers and insert them at random positions in the password
        if allow_numbers:
            for _ in range(number_count):
                number = random.choice(string.digits)
                insert_index = random.randint(0, length - 1)  # Generate a random index within the valid range
                while insert_index in used_indexes:
                    insert_index = random.randint(0, length - 1)  # Generate a new random index if already used
                password = password[:insert_index] + number + password[insert_index:]
                used_indexes.add(insert_index)

        # Convert password to lowercase if capital letters are not allowed
        if not include_capital_letters:
            password = password.lower()

        # If capital letters allowed, ensure at least 1 letter is capitalized
        if include_capital_letters:
            capital_index = random.randint(0, length - 1)  # Generate a random index within the password length
            password = password[:capital_index] + password[capital_index].upper() + password[capital_index + 1:]

        # Ensure that the password length matches the desired length
        #password = password[:length]

        return password


if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = PasswordManager()
    window.show()
    sys.exit(app.exec_())


SystemExit: 0

/Users/jjnajjar/miniconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
